In [104]:
from Function.SNNModule import *
from Function.SNN_230404 import SNNModel
from Function.plot_SNN import *
from Function.ReceptiveField_GPU import *
from Function.Stimulation_GPU import *
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import scipy.io
import time
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
print("GPU available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

# Enable GPU support if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)


GPU available: True
CUDA version: 10.1
cpu


In [106]:
pixel_h, pixel_w = 3, 1  # (#)

In [107]:
a = torch.tensor([[10, 10, 10]])
b = torch.tensor([10,10,10])

print(a,a.shape)
print(b,b.shape)

tensor([[10, 10, 10]]) torch.Size([1, 3])
tensor([10, 10, 10]) torch.Size([3])


In [108]:
receptive_field_optimized = [torch.tensor([[10, 10, 10]]),
                             torch.tensor([[10, 10, 10]])]

synaptic_delays_top = [torch.tensor([[0, 0, 0]], device=device),
                       torch.tensor([[1, 1, 1]], device=device)]

rf, rf_length = generate_afferent_to_top_rf(receptive_field_optimized, pixel_h=1,
                                            pixel_w=3, step_size=1, device=device)

rf_delay, rf_delay_length = generate_afferent_to_top_rf(synaptic_delays_top, pixel_h=1,
                                                        pixel_w=3, step_size=1, device=device)


print(rf,rf.shape)
print(rf_delay,rf_delay.shape)


Complete! Generated 2 receptive fields from afferents to the top with kernel size 1x3. with step size 0x0
Complete! Generated 2 receptive fields from afferents to the top with kernel size 1x3. with step size 0x0
tensor([[10., 10., 10.],
        [10., 10., 10.]]) torch.Size([2, 3])
tensor([[0., 0., 0.],
        [1., 1., 1.]]) torch.Size([2, 3])


In [109]:
# Create Izhikevich layers
layers = [
    IzhikevichLayer(0.02, 0.2, -65, 8,num_neurons= 2, a_decay=1, device=device),
]

# Create synapses
synapses = [
    Synapse(rf, rf_delay, device=device),
]


layers initialized
Synapses initialized


In [110]:
num_stim = 30

# Generate the stimuli and stack them up
stim = torch.zeros((pixel_h, pixel_w, num_stim), device=device)

stim[:,0,1] = 1

In [123]:
S = SNNModel(layers,synapses,device = device)

spike_times = [torch.zeros(
    (synapse.weights.shape[0], num_stim - 1)) for synapse in S.synapses]

v = [torch.zeros(
    (synapse.weights.shape[0], num_stim - 1)) for synapse in S.synapses]



for i in range(num_stim):
    # print(stim[:,:,i],stim[:,:,i].shape)
    # print(stim[:,:,i].reshape(-1),stim[:,:,i].reshape(-1).shape)
    input_current = S.synapses[0].cal_post_input(stim[:, :, i].reshape(-1))

    



Initializing SNN Model with 1 layers with device cpu
